# GLM
1. Do GLMs
2. Cluster on WT
3. Do generalization test of wt (separate GLMs) 


In [ ]:
import numpy as np
import pandas as pd
import dill as pickle
from ChangeOfMind.functions import processing as proc
from ChangeOfMind.Figures import Figure_Maker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from scipy.spatial import procrustes
from scipy.ndimage import gaussian_filter1d




datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/AllData/workspace.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)


metadata={}
metadata['base_folder']='/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/NeuronGLM/'
metadata['vars']={'x1':'speed','x2':'reldist','x3':'relspeed','x4':'reltime','x5':'wt'}
metadata['zthresh']=2.57 #99% interval
metadata['emu']={}
metadata['nhp']={}


#list usable sessions
for subject in dat['psth_sess_emu'].keys():
    metadata['emu'][subject]={}
    metadata['emu'][subject]['session']=[1]
    


Get mean tuning of neurons and tuning from GLM


In [ ]:
Figure_Maker.Fig_2_neural_glm(params={'plottype':'getFR','cred_interval':'95','model_z_thresh':2.0,'full_model':False,'folder':'/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Fig/'})

Figure_Maker.Fig_2_neural_glm(params={'plottype':'tuning','cred_interval':'95','model_z_thresh':2.0,'full_model':False,'folder':'/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Fig/'})

Visualize Geometry comparison via tuning

In [ ]:
datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/tuning_mu.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)

X=np.vstack(dat[(dat.variable=='wt') & (dat.area=='acc')].tuning_mu.values)
Y=np.vstack(dat[(dat.variable=='wt') & (dat.area=='hpc')].tuning_mu.values)

X=np.apply_along_axis(gaussian_filter1d,axis=1,arr=X,sigma=0.6)
Y=np.apply_along_axis(gaussian_filter1d,axis=1,arr=X,sigma=0.6)
                  

X=X-np.mean(X,axis=1,keepdims=True)
Y=Y-np.mean(Y,axis=1,keepdims=True)
pca=PCA(n_components=8)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
pca_X=pca.fit_transform(X.transpose())
pca_Y=pca.fit_transform(Y.transpose())
ax.scatter(pca_X[:,0],pca_X[:,1],pca_X[:,2],c=np.linspace(0,10,10),cmap='viridis')
ax.scatter(pca_Y[:,0],pca_Y[:,1],pca_Y[:,2],c=np.linspace(0,10,10),cmap='viridis',marker=">")
ax.view_init(elev=45, azim=45)
plt.show()


mtx,mty,disparity=procrustes(pca_X,pca_Y)


fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(mtx[:,0],mtx[:,1],mtx[:,2],c=np.linspace(0,10,10),cmap='viridis')
ax.scatter(mty[:,0],mty[:,1],mty[:,2],c=np.linspace(0,10,10),cmap='viridis',marker=">")
plt.show()

plt.plot(np.diagonal(cdist(X.transpose(),X.transpose()),offset=1))
plt.plot(np.diagonal(cdist(Y.transpose(),Y.transpose()),offset=1))
plt.title('fisher information')
plt.show()
#Do using GLMs
var='wt'
datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/tuning_GLM.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)

X=np.stack(dat['emu']['acc'][var])
Y=np.stack(dat['emu']['hpc'][var])


X=X-np.mean(X,axis=1,keepdims=True)
Y=Y-np.mean(Y,axis=1,keepdims=True)
pca=PCA(n_components=8)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
pca_X=pca.fit_transform(X.transpose())
pca_Y=pca.fit_transform(Y.transpose())
ax.scatter(pca_X[:,0],pca_X[:,1],pca_X[:,2],c=np.linspace(0,20,20),cmap='viridis')
ax.scatter(pca_Y[:,0],pca_Y[:,1],pca_Y[:,2],c=np.linspace(0,20,20),cmap='viridis',marker=">")
plt.show()

mtx,mty,disparity=procrustes(pca_X,pca_Y)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(mtx[:,0],mtx[:,1],mtx[:,2],c=np.linspace(0,20,20),cmap='viridis')
ax.scatter(mty[:,0],mty[:,1],mty[:,2],c=np.linspace(0,20,20),cmap='viridis',marker=">")
plt.show()

plt.plot(np.diagonal(cdist(X.transpose(),X.transpose()),offset=1))
plt.plot(np.diagonal(cdist(Y.transpose(),Y.transpose()),offset=1))
plt.title('fisher information')
plt.show()


In [ ]:
datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/AllData/workspace.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)
bin_edges = np.linspace(0, 1, 10)  # Define common bin edges

X_train={}
Y_train={}
wt_out={}
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    X_train[subj]=[]
    Y_train[subj]=[]
    wt_out[subj]={}
    psth = dat['psth_sess_emu'][subj]
    Xd = dat['Xd_sess_emu'][subj]
    wt = dat['outputs_sess_emu'][subj]
    X,Y = proc.split_for_dec(psth, Xd, wt, sess=1, split_on={'relspeed': [20,80]})
    X_train[subj]=X
    Y_train[subj]=Y
    wt_out[subj][0]=X_train[subj][0][['wt']]
    wt_out[subj][1]=X_train[subj][1][['wt']]

    wt_out[subj][0]['wt']=wt_out[subj][0].wt.apply(lambda x: 1 if 0.45 <= x <= 0.5 else (-1 if -0.5 <= x <= -0.45 else 0))
    wt_out[subj][1]['wt']=wt_out[subj][1].wt.apply(lambda x: 1 if 0.45 <= x <= 0.5 else (-1 if -0.5 <= x <= -0.45 else 0))

counts=[]
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    counts.append(np.array([np.sum(wt_out[subj][0]['wt'] == 1),np.sum(wt_out[subj][0]['wt'] == -1)]))

train_n=(np.min(np.stack(counts),axis=1).min()*0.8).astype(int)
test_n=(np.min(np.stack(counts),axis=1).min()*0.2).astype(int)

areaidx=[]
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    for neuron in Y_train[subj].keys():
        #Get brain region to sort
        if (np.where(np.char.find(dat['brain_region_emu'][subj][1][neuron], 'hpc') != -1))[0].shape[
            0] == 1:
            areaidx.append('hpc')
        elif (np.where(np.char.find(dat['brain_region_emu'][subj][1][neuron], 'acc') != -1))[0].shape[
            0] == 1:
            areaidx.append('acc')
        else:
            areaidx.append('other')
            
areaidx=np.array(areaidx) 
        

# Total decoding
scores=[]
acc_corr=[]
hpc_corr=[]

for sc in range(1000):
    print(sc)
    X_dec = []
    A_dec_train = {'0':[],'1':[]}
    A_dec_test = {'0':[],'1':[]}
    B_dec_train = {'0':[],'1':[]}
    B_dec_test = {'0':[],'1':[]}


    for _, subj in enumerate(dat['psth_sess_emu'].keys()):
        #A_1
        xtmp_a = np.random.choice(np.where(wt_out[subj][0]['wt'] == 1)[0],train_n+test_n)
        train_idx_a1 = xtmp_a[:train_n]
        test_idx_a1 = xtmp_a[train_n:]
        xtmp_b = np.random.choice(np.where(wt_out[subj][0]['wt'] == -1)[0],train_n+test_n)
        train_idx_a2 = xtmp_b[:train_n]
        test_idx_a2 = xtmp_b[train_n:]
        
        xtmp_a = np.random.choice(np.where(wt_out[subj][1]['wt'] == 1)[0],train_n+test_n)
        train_idx_b1 = xtmp_a[:train_n]
        test_idx_b1 = xtmp_a[train_n:]
        xtmp_b = np.random.choice(np.where(wt_out[subj][1]['wt'] == -1)[0],train_n+test_n)
        train_idx_b2 = xtmp_b[:train_n]
        test_idx_b2 = xtmp_b[train_n:]
        
        for i in Y_train[subj].keys():
            A_dec_train['0'].append(Y_train[subj][i][0][train_idx_a1].flatten())
            A_dec_train['1'].append(Y_train[subj][i][0][train_idx_a2].flatten())
            B_dec_train['0'].append(Y_train[subj][i][1][train_idx_b1].flatten())
            B_dec_train['1'].append(Y_train[subj][i][1][train_idx_b2].flatten())
           
    # Stack, area split, make subspace weights and correlate
    sigmaA=np.diag(0.5*((np.var(np.stack(A_dec_train['0']),axis=1)+1e-10)+(np.var(np.stack(A_dec_train['1']),axis=1))+1e-10))
    sigmaB=np.diag(0.5*((np.var(np.stack(B_dec_train['0']),axis=1)+1e-10)+(np.var(np.stack(B_dec_train['1']),axis=1))+1e-10))
    muA=np.mean(A_dec_train['0'],axis=1)-np.mean(A_dec_train['1'],axis=1)
    muB=np.mean(B_dec_train['0'],axis=1)-np.mean(B_dec_train['1'],axis=1)
    wA=(np.linalg.inv(sigmaA) @ muA)
    wB=(np.linalg.inv(sigmaB) @ muB)
    wA=muA
    wB=muB
    
    acc_corr.append(np.corrcoef(wA[np.where(areaidx=='acc')[0]],wB[np.where(areaidx=='acc')[0]])[0,1])
    hpc_corr.append(np.corrcoef(wA[np.where(areaidx=='hpc')[0]],wB[np.where(areaidx=='hpc')[0]])[0,1])


Cross-cond generalize

In [ ]:
datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/AllData/workspace.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)
bin_edges = np.linspace(0, 1, 10)  # Define common bin edges

X_train={}
Y_train={}
wt_out={}
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    X_train[subj]=[]
    Y_train[subj]=[]
    wt_out[subj]={}
    psth = dat['psth_sess_emu'][subj]
    Xd = dat['Xd_sess_emu'][subj]
    wt = dat['outputs_sess_emu'][subj]
    X,Y = proc.split_for_dec(psth, Xd, wt, sess=1, split_on={'relspeed': [20,80]})
    X_train[subj]=X
    Y_train[subj]=Y
    wt_out[subj][0]=X_train[subj][0][['wt']]
    wt_out[subj][1]=X_train[subj][1][['wt']]

    wt_out[subj][0]['wt']=wt_out[subj][0].wt.apply(lambda x: 1 if 0.45 <= x <= 0.5 else (-1 if -0.5 <= x <= -0.45 else 0))
    wt_out[subj][1]['wt']=wt_out[subj][1].wt.apply(lambda x: 1 if 0.45 <= x <= 0.5 else (-1 if -0.5 <= x <= -0.45 else 0))

counts=[]
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    counts.append(np.array([np.sum(wt_out[subj][0]['wt'] == 1),np.sum(wt_out[subj][0]['wt'] == -1)]))

train_n=(np.min(np.stack(counts),axis=1).min()*0.8).astype(int)
test_n=(np.min(np.stack(counts),axis=1).min()*0.2).astype(int)

areaidx=[]
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    for neuron in Y_train[subj].keys():
        #Get brain region to sort
        if (np.where(np.char.find(dat['brain_region_emu'][subj][1][neuron], 'hpc') != -1))[0].shape[
            0] == 1:
            areaidx.append('hpc')
        elif (np.where(np.char.find(dat['brain_region_emu'][subj][1][neuron], 'acc') != -1))[0].shape[
            0] == 1:
            areaidx.append('acc')
        else:
            areaidx.append('other')
            
areaidx=np.array(areaidx) 
        

# Total decoding
scores=[]
svc_acc=[]
svc_hpc=[]
mlp_acc=[]
mlp_hpc=[]
svc_acc_ccgp=[]
svc_hpc_ccgp=[]
for sc in range(500):
    print(sc)
    X_dec = []
    X_dec_train = []
    X_dec_test = []
    X_ccgp_test=[]
    
    for _, subj in enumerate(dat['psth_sess_emu'].keys()):
        #A_1
        xtmp_a = np.random.choice(np.where(wt_out[subj][1]['wt'] == 1)[0],train_n+test_n)
        train_idx_a = xtmp_a[:train_n]
        test_idx_a = xtmp_a[train_n:]
        xtmp_b = np.random.choice(np.where(wt_out[subj][1]['wt'] == -1)[0],train_n+test_n)
        train_idx_b = xtmp_b[:train_n]
        test_idx_b = xtmp_b[train_n:]
        
        
        xtmp_a = np.random.choice(np.where(wt_out[subj][0]['wt'] == 1)[0],train_n+test_n)
        test_idx_a_ccgp = xtmp_a
        xtmp_b = np.random.choice(np.where(wt_out[subj][0]['wt'] == -1)[0],train_n+test_n)
        test_idx_b_ccgp = xtmp_b
        
        
        
        
        for i in Y_train[subj].keys():
            X_dec_train.append(np.concatenate([Y_train[subj][i][1][train_idx_a].flatten(),
                                               Y_train[subj][i][1][train_idx_b].flatten()]).reshape(-1,1))
            
            X_dec_test.append(np.concatenate([Y_train[subj][i][1][test_idx_a].flatten(),
            Y_train[subj][i][1][test_idx_b].flatten()]).reshape(-1,1))
            
            X_ccgp_test.append(np.concatenate([Y_train[subj][i][0][test_idx_a_ccgp].flatten(),
            Y_train[subj][i][0][test_idx_b_ccgp].flatten()]).reshape(-1,1))
    
    
    X_dec_train=np.hstack(X_dec_train)
    xmu=X_dec_train.mean(axis=0)
    
    X_dec_test=np.hstack(X_dec_test)
    X_dec_train=X_dec_train-xmu
    X_dec_test=X_dec_test-xmu
    
    X_ccgp_test=np.hstack(X_ccgp_test)
    X_ccgp_test=X_ccgp_test-np.mean(X_ccgp_test,axis=0)

    Y_train_lda=np.concatenate([np.ones(train_n),np.zeros(train_n)])
    Y_test_lda=np.concatenate([np.ones(test_n),np.zeros(test_n)])
    
    ylabccgp=int(X_ccgp_test.shape[0]/2)
    Y_ccgp_lda=np.concatenate([np.ones(ylabccgp),np.zeros(ylabccgp)])
    
    svc = SVC(gamma='auto',kernel='rbf',C=3)
    
    svc.fit(X_dec_train[:,np.where(areaidx=='acc')[0]], Y_train_lda)
    svc_acc.append(svc.score(X_dec_test[:,np.where(areaidx=='acc')[0]],Y_test_lda))
    svc_acc_ccgp.append(svc.score(X_ccgp_test[:,np.where(areaidx=='acc')[0]],Y_ccgp_lda))
    
    
    svc.fit(X_dec_train[:,np.where(areaidx=='hpc')[0]], Y_train_lda)
    svc_hpc.append(svc.score(X_dec_test[:,np.where(areaidx=='hpc')[0]],Y_test_lda))
    svc_hpc_ccgp.append(svc.score(X_ccgp_test[:,np.where(areaidx=='hpc')[0]],Y_ccgp_lda))
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(X_dec_train[:,np.where(areaidx=='acc')[0]], Y_train_lda)
    mlp_acc.append(clf.score(X_dec_test[:,np.where(areaidx=='acc')[0]],Y_test_lda))
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(X_dec_train[:,np.where(areaidx=='hpc')[0]], Y_train_lda)
    mlp_hpc.append(clf.score(X_dec_test[:,np.where(areaidx=='hpc')[0]],Y_test_lda))




scores=[]
svc_acc_perm=[]
svc_hpc_perm=[]
mlp_acc_perm=[]
mlp_hpc_perm=[]

for sc in range(500):
    print(sc)
    X_dec = []
    X_dec_train = []
    X_dec_test = []
    X_ccgp_test=[]
    
    for _, subj in enumerate(dat['psth_sess_emu'].keys()):
        #A_1
        xtmp_a = np.random.choice(np.where(wt_out[subj][1]['wt'] == 1)[0],train_n+test_n)
        train_idx_a = xtmp_a[:train_n]
        test_idx_a = xtmp_a[train_n:]
        xtmp_b = np.random.choice(np.where(wt_out[subj][1]['wt'] == -1)[0],train_n+test_n)
        train_idx_b = xtmp_b[:train_n]
        test_idx_b = xtmp_b[train_n:]
        
        
        xtmp_a = np.random.choice(np.where(wt_out[subj][0]['wt'] == 1)[0],train_n+test_n)
        test_idx_a_ccgp = xtmp_a
        xtmp_b = np.random.choice(np.where(wt_out[subj][0]['wt'] == -1)[0],train_n+test_n)
        test_idx_b_ccgp = xtmp_b
        
        
        
        
        for i in Y_train[subj].keys():
            X_dec_train.append(np.concatenate([Y_train[subj][i][1][train_idx_a].flatten(),
                                               Y_train[subj][i][1][train_idx_b].flatten()]).reshape(-1,1))
            
            X_dec_test.append(np.concatenate([Y_train[subj][i][1][test_idx_a].flatten(),
            Y_train[subj][i][1][test_idx_b].flatten()]).reshape(-1,1))
            
            X_ccgp_test.append(np.concatenate([Y_train[subj][i][0][test_idx_a_ccgp].flatten(),
            Y_train[subj][i][0][test_idx_b_ccgp].flatten()]).reshape(-1,1))
    
    X_dec_train = np.apply_along_axis(np.random.permutation, axis=0, arr=X_dec_train)

    X_dec_train=np.hstack(X_dec_train)
    xmu=X_dec_train.mean(axis=0)
    
    X_dec_test=np.hstack(X_dec_test)
    X_dec_train=X_dec_train-xmu
    X_dec_test=X_dec_test-xmu
    
    X_ccgp_test=np.hstack(X_ccgp_test)
    X_ccgp_test=X_ccgp_test-np.mean(X_ccgp_test,axis=0)

    Y_train_lda=np.concatenate([np.ones(train_n),np.zeros(train_n)])
    Y_test_lda=np.concatenate([np.ones(test_n),np.zeros(test_n)])
    
    ylabccgp=int(X_ccgp_test.shape[0]/2)
    Y_ccgp_lda=np.concatenate([np.ones(ylabccgp),np.zeros(ylabccgp)])
    
    svc = SVC(gamma='auto',kernel='rbf',C=3)
    
    svc.fit(X_dec_train[:,np.where(areaidx=='acc')[0]], Y_train_lda)
    svc_acc_perm.append(svc.score(X_dec_test[:,np.where(areaidx=='acc')[0]],Y_test_lda))
    
    svc.fit(X_dec_train[:,np.where(areaidx=='hpc')[0]], Y_train_lda)
    svc_hpc_perm.append(svc.score(X_dec_test[:,np.where(areaidx=='hpc')[0]],Y_test_lda))
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(X_dec_train[:,np.where(areaidx=='acc')[0]], Y_train_lda)
    mlp_acc_perm.append(clf.score(X_dec_test[:,np.where(areaidx=='acc')[0]],Y_test_lda))
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(X_dec_train[:,np.where(areaidx=='hpc')[0]], Y_train_lda)
    mlp_hpc_perm.append(clf.score(X_dec_test[:,np.where(areaidx=='hpc')[0]],Y_test_lda))

    
    

    

    
    

In [ ]:

datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/AllData/workspace.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)
bin_edges = np.linspace(0, 1, 10)  # Define common bin edges

X_train={}
Y_train={}
wt_out={}
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    X_train[subj]=[]
    Y_train[subj]=[]
    wt_out[subj]={}
    psth = dat['psth_sess_emu'][subj]
    Xd = dat['Xd_sess_emu'][subj]
    wt = dat['outputs_sess_emu'][subj]
    Y = np.concatenate(psth[1])

    X_train[subj]=X
    Y_train[subj]=Y
    wt_out[subj][0]=X_train[subj][0][['wt']]
    wt_out[subj][1]=X_train[subj][1][['wt']]

    wt_out[subj][0]['wt']=wt_out[subj][0].wt.apply(lambda x: 1 if 0.45 <= x <= 0.5 else (-1 if -0.5 <= x <= -0.45 else 0))
    wt_out[subj][1]['wt']=wt_out[subj][1].wt.apply(lambda x: 1 if 0.45 <= x <= 0.5 else (-1 if -0.5 <= x <= -0.45 else 0))

counts=[]
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    counts.append(np.array([np.sum(wt_out[subj][0]['wt'] == 1),np.sum(wt_out[subj][0]['wt'] == -1)]))

train_n=(np.min(np.stack(counts),axis=1).min()*0.8).astype(int)
test_n=(np.min(np.stack(counts),axis=1).min()*0.2).astype(int)

areaidx=[]
for _, subj in enumerate(dat['psth_sess_emu'].keys()):
    for neuron in Y_train[subj].keys():
        #Get brain region to sort
        if (np.where(np.char.find(dat['brain_region_emu'][subj][1][neuron], 'hpc') != -1))[0].shape[
            0] == 1:
            areaidx.append('hpc')
        elif (np.where(np.char.find(dat['brain_region_emu'][subj][1][neuron], 'acc') != -1))[0].shape[
            0] == 1:
            areaidx.append('acc')
        else:
            areaidx.append('other')
            
areaidx=np.array(areaidx) 
        

# Total decoding
scores=[]
svc_acc=[]
svc_hpc=[]
mlp_acc=[]
mlp_hpc=[]
svc_acc_ccgp=[]
svc_hpc_ccgp=[]
for sc in range(500):
    print(sc)
    X_dec = []
    X_dec_train = []
    X_dec_test = []
    X_ccgp_test=[]
    
    for _, subj in enumerate(dat['psth_sess_emu'].keys()):
        #A_1
        xtmp_a = np.random.choice(np.where(wt_out[subj][1]['wt'] == 1)[0],train_n+test_n)
        train_idx_a = xtmp_a[:train_n]
        test_idx_a = xtmp_a[train_n:]
        xtmp_b = np.random.choice(np.where(wt_out[subj][1]['wt'] == -1)[0],train_n+test_n)
        train_idx_b = xtmp_b[:train_n]
        test_idx_b = xtmp_b[train_n:]
        
        
        xtmp_a = np.random.choice(np.where(wt_out[subj][0]['wt'] == 1)[0],train_n+test_n)
        test_idx_a_ccgp = xtmp_a
        xtmp_b = np.random.choice(np.where(wt_out[subj][0]['wt'] == -1)[0],train_n+test_n)
        test_idx_b_ccgp = xtmp_b
        
        
        
        
        for i in Y_train[subj].keys():
            X_dec_train.append(np.concatenate([Y_train[subj][i][1][train_idx_a].flatten(),
                                               Y_train[subj][i][1][train_idx_b].flatten()]).reshape(-1,1))
            
            X_dec_test.append(np.concatenate([Y_train[subj][i][1][test_idx_a].flatten(),
            Y_train[subj][i][1][test_idx_b].flatten()]).reshape(-1,1))
            
            X_ccgp_test.append(np.concatenate([Y_train[subj][i][0][test_idx_a_ccgp].flatten(),
            Y_train[subj][i][0][test_idx_b_ccgp].flatten()]).reshape(-1,1))
    
    
    X_dec_train=np.hstack(X_dec_train)
    xmu=X_dec_train.mean(axis=0)
    
    X_dec_test=np.hstack(X_dec_test)
    X_dec_train=X_dec_train-xmu
    X_dec_test=X_dec_test-xmu
    
    X_ccgp_test=np.hstack(X_ccgp_test)
    X_ccgp_test=X_ccgp_test-np.mean(X_ccgp_test,axis=0)

    Y_train_lda=np.concatenate([np.ones(train_n),np.zeros(train_n)])
    Y_test_lda=np.concatenate([np.ones(test_n),np.zeros(test_n)])
    
    ylabccgp=int(X_ccgp_test.shape[0]/2)
    Y_ccgp_lda=np.concatenate([np.ones(ylabccgp),np.zeros(ylabccgp)])
    
    svc = SVC(gamma='auto',kernel='rbf',C=3)
    
    svc.fit(X_dec_train[:,np.where(areaidx=='acc')[0]], Y_train_lda)
    svc_acc.append(svc.score(X_dec_test[:,np.where(areaidx=='acc')[0]],Y_test_lda))
    svc_acc_ccgp.append(svc.score(X_ccgp_test[:,np.where(areaidx=='acc')[0]],Y_ccgp_lda))
    
    
    svc.fit(X_dec_train[:,np.where(areaidx=='hpc')[0]], Y_train_lda)
    svc_hpc.append(svc.score(X_dec_test[:,np.where(areaidx=='hpc')[0]],Y_test_lda))
    svc_hpc_ccgp.append(svc.score(X_ccgp_test[:,np.where(areaidx=='hpc')[0]],Y_ccgp_lda))
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(X_dec_train[:,np.where(areaidx=='acc')[0]], Y_train_lda)
    mlp_acc.append(clf.score(X_dec_test[:,np.where(areaidx=='acc')[0]],Y_test_lda))
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(X_dec_train[:,np.where(areaidx=='hpc')[0]], Y_train_lda)
    mlp_hpc.append(clf.score(X_dec_test[:,np.where(areaidx=='hpc')[0]],Y_test_lda))



